<a href="https://colab.research.google.com/github/s-zhang-cs/Google-Colabs/blob/main/ANN_regression_codealong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From Pytorch for deep learning with Python Bootcamp by Jose Portilla


In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline